# A/B тест на трёх вариантах

В данном исследовании предстоит определить, какой из трёх вариантов маркетинговой кампании для нового продукта фаст-фуд компании работает наиболее эффективно

## Постановка гипотез

Нулевая гипотеза ($H_0$): Средние продажи во всех трёх группах одинаковы\
Альтернативная гипотеза ($H_1$): Хотя бы в одной группы продажи отличаются

## Подготовка данных

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy

In [19]:
data = pd.read_csv('WA_Marketing-Campaign.csv')
data = data.drop_duplicates()
data = data.dropna()
data.head()

,MarketID,MarketSize,LocationID,AgeOfStore,Promotion,week,SalesInThousands
0,1,Medium,1,4,3,1,33.73
1,1,Medium,1,4,3,2,35.67
2,1,Medium,1,4,3,3,29.03
3,1,Medium,1,4,3,4,39.25
4,1,Medium,2,5,2,1,27.81


In [20]:
data['MarketID'] = data['MarketID'].astype('category')
data['LocationID'] = data['LocationID'].astype('category')
data['Promotion'] = data['Promotion'] .astype('category')
data['week'] = data['week'] .astype('category')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548 entries, 0 to 547
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   MarketID          548 non-null    category
 1   MarketSize        548 non-null    object  
 2   LocationID        548 non-null    category
 3   AgeOfStore        548 non-null    int64   
 4   Promotion         548 non-null    category
 5   week              548 non-null    category
 6   SalesInThousands  548 non-null    float64 
dtypes: category(4), float64(1), int64(1), object(1)
memory usage: 21.5+ KB
